In [1]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
    return str_text

In [2]:
#read_file("moby_dick_four_chapters.txt")

In [3]:
import spacy

In [4]:
nlp = spacy.load("en", disable = ["parser", "tagger", "ner"])

In [5]:
nlp.max_length = 1198623

In [6]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [7]:
d = read_file("moby_dick_four_chapters.txt")

In [8]:
tokens = separate_punc(d)

In [9]:
len(tokens)

11338

In [10]:
train_len = 25+1
text_sequences = []
for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

In [12]:
text_sequences[0]

['call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on']

In [13]:
len(text_sequences)

11312

In [14]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [16]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [17]:
sequences[0]

[956,
 14,
 263,
 51,
 261,
 408,
 87,
 219,
 129,
 111,
 954,
 260,
 50,
 43,
 38,
 315,
 7,
 23,
 546,
 3,
 150,
 259,
 6,
 2712,
 14,
 24]

In [21]:
for i in sequences[0]:
    print(f"{i} : {tokenizer.index_word[i]}")

956 : call
14 : me
263 : ishmael
51 : some
261 : years
408 : ago
87 : never
219 : mind
129 : how
111 : long
954 : precisely
260 : having
50 : little
43 : or
38 : no
315 : money
7 : in
23 : my
546 : purse
3 : and
150 : nothing
259 : particular
6 : to
2712 : interest
14 : me
24 : on


In [22]:
tokenizer.word_counts

OrderedDict([('call', 27),
             ('me', 2471),
             ('ishmael', 133),
             ('some', 758),
             ('years', 135),
             ('ago', 84),
             ('never', 449),
             ('mind', 164),
             ('how', 321),
             ('long', 374),
             ('precisely', 37),
             ('having', 142),
             ('little', 767),
             ('or', 950),
             ('no', 1003),
             ('money', 120),
             ('in', 5647),
             ('my', 1786),
             ('purse', 71),
             ('and', 9646),
             ('nothing', 281),
             ('particular', 152),
             ('to', 6497),
             ('interest', 24),
             ('on', 1716),
             ('shore', 26),
             ('i', 7150),
             ('thought', 676),
             ('would', 702),
             ('sail', 104),
             ('about', 1014),
             ('a', 10377),
             ('see', 416),
             ('the', 15540),
             ('watery', 26),
  

In [23]:
vocab_size = len(tokenizer.word_counts)

In [24]:
vocab_size

2717

In [25]:
import numpy as np

In [26]:
sequences = np.array(sequences)

In [27]:
sequences

array([[ 956,   14,  263, ..., 2712,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2711, ...,   53,    2, 2717],
       [ 166, 2711,    3, ...,    2, 2717,   26]])

In [28]:
from keras.utils import to_categorical

In [31]:
x = sequences[:,:-1]

In [32]:
y = sequences[:,-1]

In [33]:
y = to_categorical(y, num_classes=vocab_size+1)

In [34]:
y.shape

(11312, 2718)

In [35]:
seq_len = x.shape[1]

In [36]:
seq_len

25

In [37]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [68]:
def create_model(vocab_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocab_size, seq_len, input_length=seq_len))
    model.add(LSTM(seq_len*8, return_sequences=True))
    model.add(LSTM(seq_len*8))
    model.add(Dense(seq_len*8, activation="relu"))
    model.add(Dense(vocab_size, activation="softmax"))
    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics=["accuracy"])
    model.summary()
    return model

In [69]:
model = create_model(vocab_size+1, seq_len)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 25, 25)            67950     
_________________________________________________________________
lstm_3 (LSTM)                (None, 25, 200)           180800    
_________________________________________________________________
lstm_4 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense_3 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_4 (Dense)              (None, 2718)              546318    
Total params: 1,156,068
Trainable params: 1,156,068
Non-trainable params: 0
_________________________________________________________________


In [40]:
from pickle import dump, load

In [70]:
model.fit(x,y,batch_size=128, epochs = 200, verbose=1)

C:\Users\vikra\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/200
11312/11312 [==============================] - 18s 2ms/step - loss: 6.8287 - accuracy: 0.0500
Epoch 2/200
11312/11312 [==============================] - 16s 1ms/step - loss: 6.3823 - accuracy: 0.0529
Epoch 3/200
11312/11312 [==============================] - 16s 1ms/step - loss: 6.3042 - accuracy: 0.0529
Epoch 4/200
11312/11312 [==============================] - 16s 1ms/step - loss: 6.1757 - accuracy: 0.0529 0s - loss: 6.1750 - accuracy: 0.
Epoch 5/200
11312/11312 [==============================] - 16s 1ms/step - loss: 6.0993 - accuracy: 0.0545
Epoch 6/200
11312/11312 [==============================] - 16s 1ms/step - loss: 5.9749 - accuracy: 0.0638
Epoch 7/200
11312/11312 [==============================] - 16s 1ms/step - loss: 5.8591 - accuracy: 0.0664 2s - l
Epoch 8/200
11312/11312 [==============================] - 16s 1ms/step - loss: 5.7475 - accuracy: 0.0700
Epoch 9/200
11312/11312 [==============================] - 16s 1ms/step - loss: 5.6559 - accuracy: 0.0713
Epoch 

11312/11312 [==============================] - 17s 2ms/step - loss: 2.7985 - accuracy: 0.2967
Epoch 77/200
11312/11312 [==============================] - 17s 1ms/step - loss: 2.8163 - accuracy: 0.3008
Epoch 78/200
11312/11312 [==============================] - 17s 2ms/step - loss: 2.8229 - accuracy: 0.3045
Epoch 79/200
11312/11312 [==============================] - 17s 1ms/step - loss: 2.7465 - accuracy: 0.3126
Epoch 80/200
11312/11312 [==============================] - 17s 1ms/step - loss: 2.6940 - accuracy: 0.3203
Epoch 81/200
11312/11312 [==============================] - 17s 1ms/step - loss: 2.6399 - accuracy: 0.3319
Epoch 82/200
11312/11312 [==============================] - 17s 1ms/step - loss: 2.6081 - accuracy: 0.3444
Epoch 83/200
11312/11312 [==============================] - 17s 1ms/step - loss: 2.5748 - accuracy: 0.3409
Epoch 84/200
11312/11312 [==============================] - 17s 1ms/step - loss: 2.5710 - accuracy: 0.3450
Epoch 85/200
11312/11312 [========================

11312/11312 [==============================] - 17s 1ms/step - loss: 1.1540 - accuracy: 0.7149
Epoch 152/200
11312/11312 [==============================] - 17s 1ms/step - loss: 1.1171 - accuracy: 0.7199
Epoch 153/200
11312/11312 [==============================] - 17s 1ms/step - loss: 1.0875 - accuracy: 0.7329
Epoch 154/200
11312/11312 [==============================] - 17s 1ms/step - loss: 1.0519 - accuracy: 0.7420
Epoch 155/200
11312/11312 [==============================] - 17s 1ms/step - loss: 1.0132 - accuracy: 0.7520
Epoch 156/200
11312/11312 [==============================] - 17s 2ms/step - loss: 0.9847 - accuracy: 0.7610
Epoch 157/200
11312/11312 [==============================] - 17s 2ms/step - loss: 0.9538 - accuracy: 0.7688
Epoch 158/200
11312/11312 [==============================] - 17s 2ms/step - loss: 0.9327 - accuracy: 0.7732
Epoch 159/200
11312/11312 [==============================] - 17s 2ms/step - loss: 0.8990 - accuracy: 0.7856
Epoch 160/200
11312/11312 [===============

In [71]:
model.save("model.h5")

In [72]:
dump(tokenizer, open("the_tokenizer", "wb"))

In [73]:
from keras.preprocessing.sequence import pad_sequences

In [74]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    output_text = []
    input_text = seed_text
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating="pre")
        pred_word_ind = model.predict_classes(pad_encoded, verbose = 0)[0]
        pred_word = tokenizer.index_word[pred_word_ind]
        input_text += " "+pred_word
        output_text.append(pred_word)
    return " ".join(output_text)

In [75]:
text_sequences[0]

['call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on']

In [76]:
import random
# random.seed(101)
random_pick = random.randint(0, len(text_sequences))

In [77]:
random_seed_text = text_sequences[random_pick]

In [78]:
random_seed_text

['a',
 'very',
 'appropriate',
 'little',
 'shrine',
 'or',
 'chapel',
 'for',
 'his',
 'congo',
 'idol',
 'i',
 'now',
 'screwed',
 'my',
 'eyes',
 'hard',
 'towards',
 'the',
 'half',
 'hidden',
 'image',
 'feeling',
 'but',
 'ill',
 'at']

In [79]:
seed_text = " ".join(random_seed_text)

In [80]:
seed_text

'a very appropriate little shrine or chapel for his congo idol i now screwed my eyes hard towards the half hidden image feeling but ill at'

In [81]:
generate_text(model, tokenizer, seq_len, seed_text,num_gen_words=25)

'ease meantime to see what was next to follow first he takes about a double handful of shavings out of his grego pocket and places'